## 1.2. Probability Distribution Fitting for Days to Trend Feature

This notebook contains an exploration of the target variables in the datasets on  `data/processed/`.

See [1.2 Probability Distribution Fitting](https://eddelojeda.github.io/youtube_trends/1.2_Probability_Distribution_Fitting.html) to interact with the dynamic plots in this notebook.

In [1]:
import os
import joblib
import warnings
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from youtube_trends.config import PROCESSED_DATA_DIR, MODELS_DIR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import poisson, nbinom, geom, norm, entropy, binom, zipf, logser, expon, gamma, lognorm

pio.renderers.default = 'notebook_connected'
warnings.filterwarnings('ignore')

2025-05-26 04:22:43.071 | INFO     | youtube_trends.config:<module>:11 - PROJ_ROOT path is: C:\Users\eddel\OneDrive\Documents\MCD\AAA\youtube_trends\venv\src\youtube-trends


#### Preparing training, validation, and testing datasets

Based on what was observed in the initial exploration of the data regarding the frequency distribution of the days it takes for a video to become a trend, and due to the nature of the data, which when recorded day by day, do not allow the latest data in the dataset to have a correct or reliable record of the value of this variable, we will proceed to adjust the distribution of the data considering only the data in `df_train` for reliability, `df_val` and `df_test` will be used only for visualization purposes.

In [2]:
df_train = pd.read_csv(PROCESSED_DATA_DIR / 'train_dataset.csv', low_memory=False)
df_val = pd.read_csv(PROCESSED_DATA_DIR / 'val_dataset.csv', low_memory=False)
df_test = pd.read_csv(PROCESSED_DATA_DIR / 'test_dataset.csv', low_memory=False)

In [3]:
print(df_train.shape)

(46337, 200)


Feature selection.

In [4]:
y = df_train['days_to_trend'].values
y_train, y_val = train_test_split(y, test_size=0.2, random_state=42)

Kullback-Leibler divergence function.

In [5]:
def kl_divergence(p, q):
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)
    p += 1e-10
    q += 1e-10
    return entropy(p, q)

In [6]:
def get_hist(data):
    values, counts = np.unique(data, return_counts=True)
    prob = counts / counts.sum()
    return values, prob

In [7]:
x_train, p_train = get_hist(y_train)
mean_train = np.mean(y_train)
var_train = np.var(y_train)
results = {}

Poisson distribution.

In [8]:
lambda_poisson = mean_train
poisson_probs = poisson.pmf(x_train, mu=lambda_poisson)
results['Poisson'] = kl_divergence(p_train, poisson_probs)

Geometric distribution.

In [9]:
p_geom = 1 / (1 + mean_train)
geom_probs = geom.pmf(x_train, p=p_geom)
results['Geometric'] = kl_divergence(p_train, geom_probs)

Negative binomial distribution.

In [10]:
r = mean_train**2 / (var_train - mean_train) if var_train > mean_train else 1
p_nbinom = r / (r + mean_train)
nbinom_probs = nbinom.pmf(x_train, n=r, p=p_nbinom)
results['Negative Binomial'] = kl_divergence(p_train, nbinom_probs)

Discretized Normal

In [11]:
mu_norm, sigma_norm = mean_train, np.std(y_train)
normal_probs = norm.pdf(x_train, loc=mu_norm, scale=sigma_norm)
normal_probs /= normal_probs.sum()
results['Discretized Normal'] = kl_divergence(p_train, normal_probs)

Binomial distribution.

In [12]:
n_binom = int(np.max(y_train))
p_binom = mean_train / n_binom
binom_probs = binom.pmf(x_train, n=n_binom, p=p_binom)
results['Binomial'] = kl_divergence(p_train, binom_probs)

Results.

Zipf Law.

In [13]:
x_zipf = x_train[x_train >= 1]
p_zipf = p_train[x_train >= 1]
a_zipf = 2.0
zipf_probs = zipf.pmf(x_zipf, a=a_zipf)
zipf_probs /= zipf_probs.sum()
results['Zipf'] = kl_divergence(p_zipf, zipf_probs)

Log-Series.

In [14]:
x_logser = x_train[x_train >= 1]
p_logser = p_train[x_train >= 1]
p_ls = 0.9
logser_probs = logser.pmf(x_logser, p=p_ls)
logser_probs /= logser_probs.sum()
results['Log-Series'] = kl_divergence(p_logser, logser_probs)

Discretized Log-Normal.

In [15]:
shape_ln = np.std(np.log1p(y_train))
scale_ln = np.exp(np.mean(np.log1p(y_train)))
lognorm_probs = lognorm.pdf(x_train, s=shape_ln, scale=scale_ln)
lognorm_probs /= lognorm_probs.sum()
results['Discretized Log-Normal'] = kl_divergence(p_train, lognorm_probs)

Visualization of each adjusted distribution.

In [16]:
def plot_distribution_fit(name, empirical_x, empirical_p, fitted_p):
    kl_score = kl_divergence(empirical_p, fitted_p)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=empirical_x,
        y=empirical_p,
        name='Empirical',
        opacity=0.6
    ))
    fig.add_trace(go.Scatter(
        x=empirical_x,
        y=fitted_p,
        mode='lines+markers',
        name=f'{name} (fitted)',
        line=dict(color='orange')
    ))
    fig.update_layout(
        title=f"Ajuste de {name} sobre df_train | KL Divergence = {kl_score:.4f}",
        xaxis_title="days_to_trend",
        yaxis_title="Probabilidad",
        legend=dict(x=0.99, y=0.99),
        template="plotly_white"
    )
    fig.show()

In [17]:
x_train, p_train = get_hist(y_train)
x_train = x_train.astype(int)
p_train += 1e-10

In [18]:
for dist_name in results.keys():
    if dist_name == 'Poisson':
        probs = poisson.pmf(x_train, mu=lambda_poisson)
    elif dist_name == 'Geometric':
        probs = geom.pmf(x_train, p=p_geom)
    elif dist_name == 'Negative Binomial':
        probs = nbinom.pmf(x_train, n=r, p=p_nbinom)
    elif dist_name == 'Discretized Normal':
        probs = norm.pdf(x_train, loc=mu_norm, scale=sigma_norm)
        probs /= probs.sum()
    elif dist_name == 'Binomial':
        probs = binom.pmf(x_train, n=n_binom, p=p_binom)
    elif dist_name == 'Zipf':
        x_zipf_plot = x_train[x_train >= 1]
        probs = zipf.pmf(x_zipf_plot, a=a_zipf)
        probs /= probs.sum()
        x_used = x_zipf_plot
    elif dist_name == 'Log-Series':
        x_logser_plot = x_train[x_train >= 1]
        probs = logser.pmf(x_logser_plot, p=p_ls)
        probs /= probs.sum()
        x_used = x_logser_plot
    elif dist_name == 'Discretized Exponential':
        probs = expon.pdf(x_train, scale=1/lambda_exp)
        probs /= probs.sum()
    elif dist_name == 'Discretized Gamma':
        probs = gamma.pdf(x_train, a=alpha_gamma, scale=1/beta_gamma)
        probs /= probs.sum()
    elif dist_name == 'Discretized Log-Normal':
        probs = lognorm.pdf(x_train, s=shape_ln, scale=scale_ln)
        probs /= probs.sum()
    else:
        continue
    if dist_name in ['Zipf', 'Log-Series']:
        x_plot = x_used
        p_empirical = p_train[np.isin(x_train, x_plot)]
    else:
        x_plot = x_train
        p_empirical = p_train

    probs += 1e-10
    probs /= probs.sum()
    plot_distribution_fit(dist_name, x_plot, p_empirical, probs)

In [19]:
results_sorted = sorted(results.items(), key=lambda x: x[1])
print("Ranking by KL Divergence (lower is better):")
for name, score in results_sorted:
    print(f"{name}: KL Divergence = {score:.5f}")

Ranking by KL Divergence (lower is better):
Negative Binomial: KL Divergence = 0.02657
Discretized Normal: KL Divergence = 0.09399
Log-Series: KL Divergence = 0.53236
Zipf: KL Divergence = 1.11181
Poisson: KL Divergence = 1.24942
Geometric: KL Divergence = 1.44136
Discretized Log-Normal: KL Divergence = 1.46012
Binomial: KL Divergence = 2.08831


In [20]:
best_name, best_kl = results_sorted[0]
print(f"\nBest distribution: {best_name} (KL Divergence = {best_kl:.5f})")

match best_name:
    case 'Poisson':
        y_pred_val = poisson.rvs(mu=lambda_poisson, size=len(y_val))
    case 'Geometric':
        y_pred_val = geom.rvs(p=p_geom, size=len(y_val))
    case 'Negative Binomial':
        y_pred_val = nbinom.rvs(n=r, p=p_nbinom, size=len(y_val))
    case 'Discretized Normal':
        y_pred_val = np.round(norm.rvs(loc=mu_norm, scale=sigma_norm, size=len(y_val))).astype(int)
    case 'Binomial':
        y_pred_val = binom.rvs(n=n_binom, p=p_binom, size=len(y_val))
    case 'Zipf':
        y_pred_val = zipf.rvs(a=a_zipf, size=len(y_val))
        y_pred_val = np.clip(y_pred_val, 0, np.max(y_train))
    case 'Log-Series':
        y_pred_val = logser.rvs(p=p_ls, size=len(y_val))
    case 'Discretized Exponential':
        y_pred_val = np.round(expon.rvs(scale=1/lambda_exp, size=len(y_val))).astype(int)
    case 'Discretized Gamma':
        y_pred_val = np.round(gamma.rvs(a=alpha_gamma, scale=1/beta_gamma, size=len(y_val))).astype(int)
    case 'Discretized Log-Normal':
        y_pred_val = np.round(lognorm.rvs(s=shape_ln, scale=scale_ln, size=len(y_val))).astype(int)



Best distribution: Negative Binomial (KL Divergence = 0.02657)


In [21]:
MAE = mean_absolute_error(y_val, y_pred_val)
RMSE = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"\nEvaluation in validation set:")
print(f"MAE = {MAE:.6f}")
print(f"RMSE = {RMSE:.6f}")


Evaluation in validation set:
MAE = 6.749029
RMSE = 8.823058


In [22]:
bins = np.arange(0, max(y_val.max(), y_pred_val.max()) + 2) 

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=y_val,
    xbins=dict(start=bins.min(), end=bins.max(), size=1),
    name='True value',
    opacity=0.5,
    histnorm='probability density',
))

fig.add_trace(go.Histogram(
    x=y_pred_val,
    xbins=dict(start=bins.min(), end=bins.max(), size=1),
    name='Predicted',
    opacity=0.5,
    histnorm='probability density',
    marker_color='orange'
))

fig.update_layout(
    title=f"Actual vs. predicted distribution ({best_name})",
    xaxis_title="days_to_trend",
    yaxis_title="Density",
    barmode='overlay',
    bargap=0.1,
    template='plotly_white',
    legend=dict(title=''),
)

fig.show()

In [23]:
y_val = df_val['days_to_trend'].values
match best_name:
    case 'Poisson':
        y_pred_val = poisson.rvs(mu=lambda_poisson, size=len(y_val))
    case 'Geometric':
        y_pred_val = geom.rvs(p=p_geom, size=len(y_val))
    case 'Negative Binomial':
        y_pred_val = nbinom.rvs(n=r, p=p_nbinom, size=len(y_val))
    case 'Discretized Normal':
        y_pred_val = np.round(norm.rvs(loc=mu_norm, scale=sigma_norm, size=len(y_val))).astype(int)
    case 'Binomial':
        y_pred_val = binom.rvs(n=n_binom, p=p_binom, size=len(y_val))
    case 'Zipf':
        y_pred_val = zipf.rvs(a=a_zipf, size=len(y_val))
        y_pred_val = np.clip(y_pred_val, 0, np.max(y_train))
    case 'Log-Series':
        y_pred_val = logser.rvs(p=p_ls, size=len(y_val))
    case 'Discretized Exponential':
        y_pred_val = np.round(expon.rvs(scale=1/lambda_exp, size=len(y_val))).astype(int)
    case 'Discretized Gamma':
        y_pred_val = np.round(gamma.rvs(a=alpha_gamma, scale=1/beta_gamma, size=len(y_val))).astype(int)
    case 'Discretized Log-Normal':
        y_pred_val = np.round(lognorm.rvs(s=shape_ln, scale=scale_ln, size=len(y_val))).astype(int)

In [24]:
bins = np.arange(0, max(y_val.max(), y_pred_val.max()) + 2)

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=y_val,
    xbins=dict(start=bins.min(), end=bins.max(), size=1),
    name='Valor real',
    opacity=0.5,
    histnorm='probability density',
))

fig.add_trace(go.Histogram(
    x=y_pred_val,
    xbins=dict(start=bins.min(), end=bins.max(), size=1),
    name='Valor predicho',
    opacity=0.5,
    histnorm='probability density',
    marker_color='orange'
))

fig.update_layout(
    title=f"Distribución real vs predicha en df_val ({best_name})",
    xaxis_title="days_to_trend",
    yaxis_title="Densidad",
    barmode='overlay',
    bargap=0.1,
    template='plotly_white',
    legend=dict(title=''),
)

fig.show()

In [25]:
y_test = df_test['days_to_trend'].values
match best_name:
    case 'Poisson':
        y_pred_test = poisson.rvs(mu=lambda_poisson, size=len(y_val))
    case 'Geometric':
        y_pred_test = geom.rvs(p=p_geom, size=len(y_val))
    case 'Negative Binomial':
        y_pred_test = nbinom.rvs(n=r, p=p_nbinom, size=len(y_val))
    case 'Discretized Normal':
        y_pred_test = np.round(norm.rvs(loc=mu_norm, scale=sigma_norm, size=len(y_val))).astype(int)
    case 'Binomial':
        y_pred_test = binom.rvs(n=n_binom, p=p_binom, size=len(y_val))
    case 'Zipf':
        y_pred_test = zipf.rvs(a=a_zipf, size=len(y_val))
        y_pred_test = np.clip(y_pred_test, 0, np.max(y_train))
    case 'Log-Series':
        y_pred_test = logser.rvs(p=p_ls, size=len(y_val))
    case 'Discretized Exponential':
        y_pred_test = np.round(expon.rvs(scale=1/lambda_exp, size=len(y_val))).astype(int)
    case 'Discretized Gamma':
        y_pred_test = np.round(gamma.rvs(a=alpha_gamma, scale=1/beta_gamma, size=len(y_val))).astype(int)
    case 'Discretized Log-Normal':
        y_pred_test = np.round(lognorm.rvs(s=shape_ln, scale=scale_ln, size=len(y_val))).astype(int)

In [26]:
y_test = df_test['days_to_trend'].values
bins = np.arange(0, max(y_test.max(), y_pred_test.max()) + 2)

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=y_test,
    xbins=dict(start=bins.min(), end=bins.max(), size=1),
    name='Valor real',
    opacity=0.5,
    histnorm='probability density',
))

fig.add_trace(go.Histogram(
    x=y_pred_test,
    xbins=dict(start=bins.min(), end=bins.max(), size=1),
    name='Valor predicho',
    opacity=0.5,
    histnorm='probability density',
    marker_color='orange'
))

fig.update_layout(
    title=f"Distribución real vs predicha en df_val ({best_name})",
    xaxis_title="days_to_trend",
    yaxis_title="Densidad",
    barmode='overlay',
    bargap=0.1,
    template='plotly_white',
    legend=dict(title=''),
)

fig.show()

**Note:** As expected, the results of the recently published videos are not very accurate because the values ​​of this variable in the dataset probably have not yet reached their true value.

Saving model.

In [27]:
model_info = {
    'distribution': best_name,
    'kl_divergence': best_kl,
}

In [28]:
match best_name: 
    case 'Poisson':
        model_info['params'] = {'mu': lambda_poisson}
    case 'Geometric':
        model_info['params'] = {'p': p_geom}
    case 'Negative Binomial':
        model_info['params'] = {'n': r, 'p': p_nbinom}
    case 'Discretized Normal':
        model_info['params'] = {'mu': mu_norm, 'sigma': sigma_norm}
    case 'Binomial':
        model_info['params'] = {'n': n_binom, 'p': p_binom}
    case 'Zipf':
        model_info['params'] = {'a': a_zipf}
    case 'Log-Series':
        model_info['params'] = {'p': p_ls}
    case 'Discretized Exponential':
        model_info['params'] = {'lambda': lambda_exp}
    case 'Discretized Gamma':
        model_info['params'] = {'alpha': alpha_gamma, 'beta': beta_gamma}
    case 'Discretized Log-Normal':
        model_info['params'] = {'s': shape_ln, 'scale': scale_ln}

In [29]:
joblib.dump(model_info, MODELS_DIR / 'distribution_days_trend.pkl')
print(f'Model saved as "distribution_days_trend.pkl" in {MODELS_DIR}.')

Model saved as "distribution_days_trend.pkl" in C:\Users\eddel\OneDrive\Documents\MCD\AAA\youtube_trends\venv\src\youtube-trends\models.
